In [1]:
!nvidia-smi

Sun Jul  9 14:21:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch--2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install einops

ERROR: Could not find a version that satisfies the requirement torch--2.0.1 (from versions: none)
ERROR: No matching distribution found for torch--2.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.6 MB/s eta 0:00:00


In [6]:
import os
from pprint import pprint
import json

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /opt/conda/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib'), PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/lib/x86_64-linux-gnu')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_tra

In [8]:
MODEL_NAME = "tiiuae/falcon-7b-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model =AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [10]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [11]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


In [91]:
!pip install pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [92]:
import pandas as pd

In [95]:
df = pd.read_json("/kaggle/input/aibe-mcq/100_questions.json")

In [104]:
df["Question"][0]

'1. Section 66A of Information Technology Act was held unconstitutional in the case of'

In [105]:
df["Options"][0]

['a) Justice K. S. Putta swamy Vs Union of India',
 'b) M P Sharma Vs Satish Chandra',
 'c) Shreya Singhal Vs Union Of India',
 'd) Gagan Harsh Sharma Vs The State of Maharashtra']

In [103]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 256,
  "pad_token_id": 11,
  "top_p": 0.7,
  "transformers_version": "4.30.0.dev0"
}

In [98]:
generation_config = model.generation_config
generation_config.max_new_tokens = 256
generation_config.temperature = 1
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

device = "cuda:0"

In [102]:
for i in range(0,1):
    prompt = "Question " + df["Question"][i] + "\n" + "Options " + str(df["Options"][i])
    prompt.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
      outputs = model.generate(
          input_ids=encoding.input_ids,
          attention_mask=encoding.attention_mask,
          generation_config=generation_config,
      )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)
    last_line = response.splitlines()[-1]
    df["Unprompted_prediction"] = last_line

In [106]:
df.head(1)

,Question,Options,Unprompted_prediction
0,1. Section 66A of Information Technology Act w...,[a) Justice K. S. Putta swamy Vs Union of Indi...,d) Gagan Harsh Sharma Vs The State of Maharashtra


In [110]:
%%time
for i in range(1,6):
    prompt = "Question " + df["Question"][i] + "\n" + "Options " + str(df["Options"][i])
    prompt.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
      outputs = model.generate(
          input_ids=encoding.input_ids,
          attention_mask=encoding.attention_mask,
          generation_config=generation_config,
      )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)
    last_line = response.splitlines()[-1]
    df["Unprompted_prediction"][i] = last_line

CPU times: user 2min 7s, sys: 61.6 ms, total: 2min 7s
Wall time: 2min 7s


In [111]:
df.head(6)

,Question,Options,Unprompted_prediction
0,1. Section 66A of Information Technology Act w...,[a) Justice K. S. Putta swamy Vs Union of Indi...,d) Gagan Harsh Sharma Vs The State of Maharashtra
1,2. A Teacher is not a workman within the purvi...,[a) The Workmen Vs Greaves Cotton & Co. Ltd. &...,d) Dinesh Sharma and Ors. Vs State of Bihar
2,3. According to Factories Act,"[a) ""child"" means a person who has not complet...","a) ""child"" means a person who has not complete..."
3,4. The UNCITRAL Model Law and Rules do not bec...,[a) Union of India Vs East Coast Boat Builders...,The UNCITRAL Model Law and Rules do not become...
4,5. According to Section 7(4) of the Arbitratio...,"[a) a document signed by the parties;, b) an e...",c) an exchange of statements of claim and defe...
5,6. Waiver of right to object deviance from arb...,"[a) Section 7, b) Section 4, c) Section 20, d)...",a) Section 7


In [113]:
%%time
for i in range(6,26):
    prompt = "Question " + df["Question"][i] + "\n" + "Options " + str(df["Options"][i])
    prompt.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
      outputs = model.generate(
          input_ids=encoding.input_ids,
          attention_mask=encoding.attention_mask,
          generation_config=generation_config,
      )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)
    last_line = response.splitlines()[-1]
    df["Unprompted_prediction"][i] = last_line

CPU times: user 3min 28s, sys: 76.8 ms, total: 3min 29s
Wall time: 3min 29s


In [115]:
%%time
for i in range(26,56):
    prompt = "Question " + df["Question"][i] + "\n" + "Options " + str(df["Options"][i])
    prompt.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
      outputs = model.generate(
          input_ids=encoding.input_ids,
          attention_mask=encoding.attention_mask,
          generation_config=generation_config,
      )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)
    last_line = response.splitlines()[-1]
    df["Unprompted_prediction"][i] = last_line

CPU times: user 12min 25s, sys: 325 ms, total: 12min 25s
Wall time: 12min 26s


In [117]:
%%time
for i in range(56,86):
    prompt = "Question " + df["Question"][i] + "\n" + "Options " + str(df["Options"][i])
    prompt.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
      outputs = model.generate(
          input_ids=encoding.input_ids,
          attention_mask=encoding.attention_mask,
          generation_config=generation_config,
      )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)
    last_line = response.splitlines()[-1]
    df["Unprompted_prediction"][i] = last_line

CPU times: user 9min 55s, sys: 356 ms, total: 9min 56s
Wall time: 9min 56s


In [119]:
%%time
for i in range(86,100):
    prompt = "Question " + df["Question"][i] + "\n" + "Options " + str(df["Options"][i])
    prompt.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
      outputs = model.generate(
          input_ids=encoding.input_ids,
          attention_mask=encoding.attention_mask,
          generation_config=generation_config,
      )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)
    last_line = response.splitlines()[-1]
    df["Unprompted_prediction"][i] = last_line

CPU times: user 1min 25s, sys: 54.4 ms, total: 1min 25s
Wall time: 1min 25s


In [120]:
df["Unprompted_prediction"].to_list()

['d) Gagan Harsh Sharma Vs The State of Maharashtra',
 'd) Dinesh Sharma and Ors. Vs State of Bihar',
 'a) "child" means a person who has not completed his fifteenth year of age',
 'The UNCITRAL Model Law and Rules do not become part of the Arbitration Act so as to become an aid to construe the provisions of the Act.',
 'c) an exchange of statements of claim and defence in which the existence of the agreement is alleged by one party and not denied by the other.',
 'a) Section 7',
 'c) Section 115',
 'The Arbitration Act 1996 repeals the Arbitration (Protocol and Convention) Act, 1937.',
 'd) Plea bargaining',
 "Answer: 'c) Section 111 of Cr.P.C.'",
 "Options ['a) A can, under Section 43 para 1', 'b) A can under Section 49 para 1', 'c) A cannot and will have to wait till X,Y become traceable', 'd) Z can be compelled only for one third']",
 'a) Agency',
 '(d) Specific performance with regard to contracts',
 'a) Article 323B',
 "Answer: 'c) Section 17 of the Advocate Act, 1961'",
 'c) the

In [121]:
# Save the DataFrame to a CSV file
df.to_csv('AIBE_XV_v1_output.csv', index=False)

In [123]:
question_list = df["Question"].to_list()

In [124]:
# Initialize variables
total_length = 0
total_words = 0

# Iterate over each string in the list
for string in question_list:
    # Split the string into words
    words = string.split()
    
    # Iterate over each word
    for word in words:
        total_length += len(word)
        total_words += 1

# Calculate average word length
average_length = total_length / total_words

# Print the average word length
print("Average word length:", average_length)


Average word length: 4.7904845045831514
